<a href="https://colab.research.google.com/github/Euan-J-Austin/Analytics_and_Intelligence/blob/main/RandomForestChallenge.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

1. Prepare data set
2. Create RF regressor (y is continuous)
3. Measure accuracy
4. Tune model
5. Generate insights

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import ParameterGrid
from sklearn.ensemble import RandomForestRegressor

In [6]:
df = pd.read_csv('/content/marketing_campaign.csv')

In [7]:
df.head(3)

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
0,0,5524,1957,Graduation,Single,58138.0,0,0,04/09/2012,58,...,10,4,7,0,0,0,0,0,0,1
1,1,2174,1954,Graduation,Single,46344.0,1,1,08/03/2014,38,...,1,2,5,0,0,0,0,0,0,0
2,2,4141,1965,Graduation,Together,71613.0,0,0,21/08/2013,26,...,2,10,4,0,0,0,0,0,0,0


In [8]:
df_na = df[df.isna().any(axis=1)]

In [9]:
df_na

,Unnamed: 0,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,...,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Response
10,10,1994,1983,Graduation,Married,NaN,1,0,15/11/2013,11,...,0,2,7,0,0,0,0,0,0,0
27,27,5255,1986,Graduation,Single,NaN,1,0,20/02/2013,19,...,0,0,1,0,0,0,0,0,0,0
43,43,7281,1959,PhD,Single,NaN,0,0,05/11/2013,80,...,3,4,2,0,0,0,0,0,0,0
48,48,7244,1951,Graduation,Single,NaN,2,1,01/01/2014,96,...,1,4,6,0,0,0,0,0,0,0
58,58,8557,1982,Graduation,Single,NaN,1,0,17/06/2013,57,...,0,3,6,0,0,0,0,0,0,0
71,71,10629,1973,2n Cycle,Married,NaN,1,0,14/09/2012,25,...,0,3,8,0,0,0,0,0,0,0
90,90,8996,1957,PhD,Married,NaN,2,1,19/11/2012,4,...,2,8,9,0,0,0,0,0,0,0
91,91,9235,1957,Graduation,Single,NaN,1,1,27/05/2014,45,...,0,2,7,0,0,0,0,0,0,0
92,92,5798,1973,Master,Together,NaN,0,0,23/11/2013,87,...,4,8,1,0,0,0,0,0,0,0
128,128,8268,1961,PhD,Married,NaN,0,1,11/07/2013,23,...,1,7,6,0,0,0,0,0,0,0


In [10]:
#deal with missing values .. no income is retired, no responded?
#2n cycle to master

# Imputing missing data

In this case, it is the income values.

Will try MissForest, MiceForest, KNN ... expecting that MissForest will be superior.

Will have to enact in JupyterNotebooks locally, save results and upload to this notebook